# Access Paths


## Path Walking

The Access Path is a mechanism for, and artifact of, path walking. Each successful lookup of a path segment results in an Access Path object.

The Access Path object is an abstraction of two patterns.

    - The Access Frame
    - The Access Frame Stack
    
The Access Frame is the state of the lookup environment at each path segment.

The Access Frame Stack is the current frame, preceded by previous frames. The stack results in a pathway, from root, to target.

An Access Frame has a pointer to the previous frame. This linked list of Access Frames forms the stack. The python class APath creates an instance of an Access Frame, which by reference includes previous frames, the whole path. So one can use an APath object to represent the whole path, so it's called Access Path for convenience, instead of Access Frame.


## Environment Cascade

Each Access Frame inherits the environment of the previous frame and may overload the environment from its own content.

The APath class has a class variable `APath._rootcosm` for a default environment, which root level access paths inherit upon creation.

Root level access paths are created without a previous Access Frame. Branch Access Paths are created during path lookup.

## User Identity

The first frame in an Access Path's frame-stack is the Identity Frame.

    raygan@raygan.com

An entity must first authenticate to the Identity Frame before accessing the root of the path.

The second frame, the path-root, or Home Frame. Is the root of an entity's file system. This is the most top level of access that an entity is granted, and the starting point of all access, such as a User's personal device.

    Raygan's device
    
The personal device context has a user agent such as a web browser or a file system client. 

    --(firefox)-->

Using one of these generates the next frame, which would be the root of a file system, either on disk, or via URL.

    /
    -or-
    https://example.com/
    



## Code Execution

An APath object acts as a containerized shell during path walking and file creation. It loads and executes code to operate on the current target, extending lookup capabilities beyond directory-based lookup, such as parsing a file, searching a database, or generating data on the fly.

Code and data are accessed through Dcels. The Access Path has a `target` Dcel which it operates on. 

Resources from the Cascading Environment are retrieved as Dcels.

An executable resource, represented by a Dcel, is loaded via `APath.load_function()`. This method mimics some of the functionality of linux's `execva()`. It attempts to load the requested target via modular executable-handlers. When a handler returns a function on success, `load_function()` caches it in an access path branch. The next time the Dcel is requested for execution, the cache can be checked for the function handle.

`APath.command()` is a concise way of "running" a command. This method uses `load_function()` to generate a Dcel with the function and args stored as the Dcel's _formula_. The execution is therewith deferred until the Dcel's value is referenced, and Dcels in the args are expanded at each execution.

The result Dcel is stored in an APath branch.


## Relationship with Domain Cels (Dcels)

See `Dcel`

Domain Cels (Dcels) represent any "addressable" data set. They have an `address` and a `service`. The Dcel object provides a consistent API for interacting with the contents of a Dcel no matter how the data is stored or generated.

A Dcel provides a file-system interface into it's data via it's service module. With a service module, API's for directory lookup, file reads/writes and new file creation are standardized.

An Access Path, APath object, is associated with one Dcel for content, and one Dcel for environment. The Dcel for environment is created by a series of inherited contexts from the Access Frame Stack. The Dcel for content is created by the output of the *previous* path segment lookup within the *previous* Access Frame context.





## Bootstrapping Each Step of the Environment Cascade

#### Root

The class environment is used on the root apath to parse its' domain.

Out of the domain, is pulled up the root environment configuration and is overlayed onto the class environment of the root apath.

The root env re-parses the root env. --> Final root env.

#### Branch

The parent environment is used to parse a branch.

Out of the branch, is pulled up the branch environment configuration and is overlayed onto the parent environment of the root path.

The branch env re-parses the branch env. --> Final branch env.







## Constructor

Conditions under which an Access Path is created.

- Beginning access
- Spawned path-walk segment
- Unwrapping a wrapper such as Fudge

### Beginning access

Generate a Dcel and set as target.

    a = APath(dcel_instr, service)

### Spawning a path-walk segment

Looking up a segment.

    branch = APath(lookup_instr,
                   target,
                   parent=self)
    
### Unwrapping a wrapper

    a = APath('.')
    wrapper = APathWrapper(a)
    
If the wrapper is a subclass of APathWrapper, the `__new__` method returns the original APath instance.

    aa = APath(wrapper)
    
    aa is a


## Init

### Constructor Condition

- new
- spawn
- unwrap


### Inherit Environment

#### Stage 1

The Access Path inherits from the class.

#### Stage 2

The Access Path inherits from a _parent_.

In _Stage 2_ the `.cosm` from the parent replaces the one from the class.

#### Stage 3

The tools and services from the parent `.cosm` are used to parse the target and retrieve the local `.cosm`.

The local `.cosm` is layered ontop of the one from the parent. (If no parent, from the class.)




In [1]:
from fs.base import FS
from fs.osfs import OSFS
from fs.multifs import MultiFS
from MulticelFS import MulticelFS
from Dcel import Dcel
from Dcel import Dcel as D
from DictFS import DictFS
from FormulaFS import FormulaFS
from ApathRootCosm import apathRootCosm
from fs.errors import ResourceNotFound
from urllib.parse import urlparse

METADIRNAME = ".cosm"


class APathWrapper:
    """
    Inherit this class before wrapping an Apath.
    
    Subclasses can be unwapped by APath.
    """
    
    def __init__(self, apath):
        self._apath = apath
        

class APath(FS):
    
    # Any APath created without
    # a parent inherits the class
    # environment.
    _rootcosm = Dcel(address=apathRootCosm,
                  service_class=DictFS,
                  args=['r'],
                  writeable=False
                 )
    _rootcosm_atdir = Dcel({'@':apathRootCosm}, service_class=DictFS)
        
    def __new__(cls,
                addr=None,
                servicename=None,
                parent=None
                ):
        """
        Check to see if we can
        unwrap an Apath from
        a subclass of APathWrapper.
        Otherwise, create a new instance.
        """
        if issubclass(addr.__class__,
                      APathWrapper):
            return addr._apath

        return super().__new__(cls)
    
    
    def __init__(self,
                 addr=None,
                 servicename=None,
                 parent=None
                ):
        
        # Constructor Conditions
        # 1. Unwrap
        # 2. Begin Access
        # 3. Spawn Branch
        
        ## Unwrap
        #
        try:
            """
            In this case, the Apath
            class might be unwrapping
            an Apath from a wrapper.
            """
            if self.__inited == True:
                return
        except AttributeError:
            # a new instance.
            pass
        
        ## Init new instance
        super().__init__()
        
        ## Inherit rootcosm environment from class
        #  set .cosm (cascading operating system)
        #  .cosm is a Dcel or PyFilesystem2 FS subclass
        # THE COSM WILL BE OVERLOADED AT THE END OF THIS METHOD
        # BUT WE NEED THIS TO PROVIDE THE SERVICES FOR INIT.
        if parent is None:
            self.cosm = APath._rootcosm
        else:
            self.cosm = parent.cosm
        
        ## Init directory 
        self.branch = dict()
        
        ## Set inited flag
        self.__inited = True
        
        ## Beginning Access
        #  addr is a url
        #  service is None
        if type(addr) is str and servicename is None:
            url = urlparse(addr)
            servicename = url.scheme
        
        #  addr
        #  service
        if type(servicename) is str:
            service = self.cosm['services'][servicename].value
            self.target = Dcel(address=addr, 
                               service_class=service)   
        
        ## Spawning a Branch
        #  Dcel
        #  Parent Apath
        elif type(addr) is Dcel:
            # WIP lookup instruction
            self.target = addr
            
            ## inherit from parent
            if parent is None:
                # create a readonly cel for
                # root cosm
                pass
            # else
            # cascade the parent .cosm
            # TODO
            
        # parent
        self.parent = parent
        

            
        ## Inherit rootcosm environment from class
        #  set .cosm (cascading operating system)
        #  .cosm is a Dcel or PyFilesystem2 FS subclass
        
        # - read '@' and '.@' entries
        # - cascade '@' and '.@' onto .cosm
        # Quick Hack:
        # - Make a fs.multifs::MultiFS of:
        #        parent.target Dcel, write=False
        #        this.target Dcel, write=True
        # - Make a fs.subfs::SubFS via opendir() of:
        #        theAboveResult.opendir('@')
        # - The resulting SubFS instance will allow writes
        #   only if '@' exists in the writable layer
        #   AND WHEN the @ comes into existence writes become available.
        
        # V2
        
        # Use ApathRootCosm['etc']['cosmdirname']
        # to customize variants of cosm directory.
        # Precedence is determined by list order.
        cosmdirnamelist = self.cosm.listdir('/etc/cosmdirname')
        
        if parent is None:
            parent_cosm = APath._rootcosm
        else:
            parent_cosm = parent.cosm
        
        if self.target.isdir():
            try:
                cosmstack = [ Dcel(parent_cosm, writeable=False),
                             *[Dcel(self.target[ea], writeable=False)
                               for ea in self.target.listdir()
                               if ea in cosmdirnamelist
                              ]
                            ]  
            except:
                cosmstack = [ Dcel(parent_cosm, writeable=False) ]
        else:
            cosmstack = [ Dcel(parent_cosm, writeable=False) ]
        self.cosm = Dcel(cosmstack, service_class=MulticelFS)     
        
    def _reinit(self,
                 addr=None,
                 servicename=None,
                 parent=None
                ):
        self.branch = dict()
        self.__inited = True
        
        ## Beginning Access
        #  addr is a url
        #  service is None
        if type(addr) is str and servicename is None:
            url = urlparse(addr)
            servicename = url.scheme
        
        #  addr
        #  service
        if type(servicename) is str:
            service = self.cosm['services'][servicename].value
            self.target = Dcel(address=addr, 
                               service_class=service)   
        
        ## Spawning a Branch
        #  Dcel
        #  Parent Apath
        elif type(addr) is Dcel:
            # WIP lookup instruction
            self.target = addr
            
            ## inherit from parent
            if parent is None:
                # create a readonly cel for
                # root cosm
                pass
            # else
            # cascade the parent .cosm
            # TODO
            
        # parent
        self.parent = parent
        
        # overlay local environment .cosm
        # TODO
    
    def _inherit_cosm(self):
        
        # get cosm dcel from parent
        par_cosmcel = self.parent.cosm
        
        # get cosm dcel from target
        lookupfn = self.parent.cosm['sbin']['lookup']
        targ_cosmcel = self.target['.cosm']
        
        # create new dcel from both
        d = Dcel(address=[par_cosmcel, targ_cosmcel],
              service_class=CascadeCelFS)
    
    def _spawn(parent, name, dcel):
        # works as classy or self method
        ap = APath(dcel, parent=parent)
        parent.branch[name] = ap
        return ap
    
    def close(self):
        # TODO:
        # need to close _cosm.fs[*]?
        # close DCel?
        super().close()
    
    def detect_service(self,addr):
        if addr is dict:
            return self.cosm['services']['dict'].value
        return self.cosm['services']['file']
    
    def start_service(self,addr,service=None):
        if service is None:
            _Service = self.detect_service(addr)
        else:
            _Service = service
        return _Service(addr)
    
    
    # lookup interface --------------
    
    def _pathwalk(self,path=""):
        if path in (None,"",".","/"):
            return self
        path = path.strip("/")
        try:
            seg,nextpath = path.split('/',1)
        except:
            return self.lookup(path)
        try:
            return self.lookup(seg)._pathwalk(nextpath)
        except:
            raise ResourceNotFound(path)
    
    def path_lookup(self,path):
        return self._pathwalk(path)
        
    def lookup(self, addr):
        """ 
        Return an entry of the current
        target's directory
        wrapped in an Apath.
        """
        try:
            d = self.target[addr]
            try:
                a = self.branch[addr]
                if a.target != d:
                    a = APath(d,parent=self)
                    self.branch[addr] = a
                return a
            except:
                a = APath(d,parent=self)
                self.branch[addr] = a
                return a
        except:
            try:
                a = self.branch[addr]
                return a
            except:
                raise
            raise
    
    def getchild(self, key):
        return self.lookup(key)
    
    def exists(self, key):
        try:
            self.lookup(key)
            return True
        except:
            return False
    
    def getprop(self, key):
        # test version only returns self
        return self
    
    # execution interface -----------
    
    def canonical_cmdline(cmdname, *args, **kwargs):
        # fixme: include args and kwargs
        return cmdname
    
    def load_function(self, fncel:Dcel):
        return fncel.value
    
    def resolve_cmdpath(self, cmdname):
        for p in self.cosm['env']['PATH'].value:
            # lookup command dcel
            _p = p.removeprefix('.cosm/')
            pathname = _p+'/'+cmdname
            try:
                cmdcel = self.cosm.path_lookup(pathname)
            except:
                continue
            if not cmdcel is None:
                break
        return cmdcel
    
    def resolve_interpreter(self, cmdcel):
        val = cmdcel.value
        if type(val) is dict:
            try:
                interp_path = val['#!'][0]
            except:
                return (None,[])
            try:
                interp_args = val['#!'][1:]
            except:
                interp_args = []
            interp_cel = self.resolve_cmdpath(interp_path)
            return (interp_cel,interp_args)
        return (None,[])

    def command(self, rtntype, cmdname, *args, **kwargs):        
        # use cache
        cmd_id = self.canonical_cmdline(cmdname, *args, **kwargs)
        #try:
          #  return self.branch[cmd_id]      
        #except:
            #pass
        if str(type(cmdname))=="<class 'function'>":
            fn = cmdname
        else:
            cmdcel = self.resolve_cmdpath(cmdname)
            interp_cel,interp_args = self.resolve_interpreter(cmdcel)
            #print(f"APath::command: interp_cel: {interp_cel.inspect()}")
            print(f"APath::command: interp_args: {interp_args}")
            # get function pointer
            if interp_cel != None:
                fn = self.load_function(interp_cel)
                args = interp_args + [cmdcel] + [*args]
            else:
                fn = self.load_function(cmdcel)

        # set service for return type
        if issubclass(rtntype,FS):
            service_cls = rtntype
        else:
            service_cls = self.detect_service(rtntype)
        
        # create dcel with formula
        # Using 2024 January FormulaFS
        
        dc = Dcel({'fn':fn,
                   'args':[*args],
                   'kwargs':kwargs
                  },
                 service_class=FormulaFS)
        
        # older style formula Dcel
        # requires internal formula-aware logic
        # inside the Dcel object, which is hard to maintain.
        #dc = Dcel( formula=fn,
         #          args=args,
          #         kwargs=kwargs,
           #      )
        
        # create service wrapper
        # Q: do we need this?
        # A: Yes. The Dcel FS API (ex. `listdir()`)
        #    used by APath requires it.
        _dc = Dcel(address=dc, 
                   service_class=service_cls)
        
        # wrap in APath
        res = APath(_dc, parent=self)

        # stash
        self.branch[cmd_id] = res
        
        return res
    
    # pyfilesystem 'FS' interface ---
    
    def getinfo(self, addr=None, namespaces=None):
        _apath = self.path_lookup(addr)
        return _apath.target.getinfo(namespaces)
    
    def listdir(self,path=None):
        _apath = self.path_lookup(path)
        return _apath.target.listdir()
    
    def isdir(self,path=None):
        return self.target.isdir(path)
    
    def makedir(self,path,permissions=None,recreate=False):
        return self.target.makedir(path,permissions,recreate)
    
    def openbin(self,
            path=None,
            mode='r',
            buffering=-1):
        # Open a binary file.
        return self.target.openbin(
            path,
            mode,
            buffering)
    
    def readbytes(self, path=None):
        return self.target.readbytes(path)
        
    def readtext(self, path=None):
        return self.target.readtext(path)
        
    def remove(self,path): # Remove a file.
        return self.target.remove(path)
    
    def removedir(self,path): # Remove a directory.
        return self.target.removedir(path)
        
    def setinfo(self,path,info):  # Set resource information.
        return self.target.setinfo(path, info)


In [29]:
## Debug: Why is Fudge running 18 executions of a command?
from DictFS import DictFS

In [30]:
dummy_dcel = Dcel(address={'a':{'Aye':'A'}}, service_class=DictFS)
test_apath = APath(dummy_dcel)

# this executes 9 times
print('Running test_apath.command() as mutation.')
mutation = test_apath.command(dict, 'hello', dummy_dcel)

# Problem: `.command` should only setup the command for execution
# not execute it.

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'dict'>
Running test_apath.command() as mutation.
APath::command: interp_args: []
DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'Dcel.Dcel'>
hello.


In [4]:
print(mutation.target)

hello there.


In [5]:
## Test `getinfo()`
from fs import open_fs
a = APath(Dcel('demo-files/fs', service_class=open_fs, writeable=False))
a.listdir()

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>


['fruit', 'hello.txt', '@', 'test.txt']

In [6]:
a.listdir('/fruit')
a.getinfo('/fruit/apples.txt').name

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>


'apples.txt'

In [7]:
## Test cmdcel lookup
for _p in a.cosm['/env']['PATH'].value:
    print(_p)

.cosm/types/
.cosm/tools/
.cosm/bin/


In [8]:
## Test `filterdir()`
#  In Dev: Not working
#for each in a.filterdir(''):
#    print(each)

In [9]:
## Test `tree()`
# depends on `filterdir()`
# depends on Dcel::abspath()
a.tree()

`-- error (expected str, bytes or os.PathLike object, not tuple)


In [10]:
## Develop: Cosm inheritance
# Prepare the 'cosmstack' before handing to MulticelFS
from fs import open_fs
from Dcel import Dcel
from MulticelFS import MulticelFS
a = Dcel(address='fs/fs', service_class=open_fs, writeable=False)
cosmdirnamelist = ['.cosm','_cosm','@','_@','.@']
parent_cosm = APath._rootcosm

# == SNIP: insert into Dcel.__init__() method ==
cosmstack = [ Dcel(parent_cosm, writeable=False),
                     *[Dcel(a[ea], writeable=False)
                       for ea in a.listdir('.')
                       if ea in cosmdirnamelist
                      ]
                    ]  
m = MulticelFS(cosmstack)
# --------
m.listdir('.')

DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'dict'>


['tools',
 'hosts',
 'kernel',
 'var',
 'services',
 'machines',
 'etc',
 'bin',
 'types',
 'env']

In [11]:
cosmdirnames = ['.cosm','_cosm','@','_@','.@']
cosmdirnames_appended = [ '.newname', *[ each for each in cosmdirnames] ]
print(cosmdirnames_appended)

['.newname', '.cosm', '_cosm', '@', '_@', '.@']


In [12]:
from fs import open_fs
from Dcel import Dcel
a = Dcel(address='fs/fs', service_class=open_fs, writeable=False)
a.listdir('/')
cosmdirnames = ['.cosm','_cosm','@','_@','.@']
cosmdirs = [ a[ea] for ea in a.listdir('/') if ea in cosmdirnames ]
for ea in cosmdirs:
    print(ea.getinfo().raw['basic']['name'])


.cosm


## Setup Test Environment

In [2]:
from blackstrap import BlackstrapFS
BlackstrapFS.initHost('localhost')
#BlackstrapFS.addShare('fs/cburnwebui','cburn')
BlackstrapFS.addShare('demo-files/fs','fs')
BlackstrapFS.addShare('demo-files/fs2','fs2')


blackstrap.BlackstrapFS

## Test: `__init__()`

In [3]:
a = APath('file://fs.localhost','file')
aa = APath('file://fs2.localhost','file')

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>


In [4]:
print(a)
print(aa)

## Test `listdir()` Method

In [5]:
# TEST: listdir('/')
print(a.listdir('@'))
print(a.listdir('@/etc'))

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
['etc']
DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
['fstab']


## Test `.cosm` Property

In [6]:
print(a.cosm.listdir('.'))

DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'dict'>
['env', 'types', 'tools', 'kernel', 'bin', 'machines', 'services', 'etc']


In [7]:
targ = a.lookup('fruit')
print(targ.cosm.listdir('.'))
print(targ.listdir('.'))

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'dict'>
['env', 'types', 'kernel', 'services', 'bin', 'machines', 'tools', 'etc']
['apples.txt']


## Test `lookup()` Method

In [8]:
path = 'fruit'
targ = a.lookup('fruit')
print(targ.listdir('.'))

['apples.txt']


## Test `.command()` Method

In [9]:
fstab_file = a.lookup('@/etc/fstab')
print(fstab_file)

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>


In [10]:
# run it as by __getattr__() in Fudge.py
res = a.command(dict,'fstab',fstab_file.target)

APath::command: interp_args: []
DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'Dcel.Dcel'>


In [11]:
res.target.service.fsdict.service._formula_dict

{'fn': <function HienaMP.hiena_mp(g: dict, text: Dcel.Dcel, rulename='$__start__')>,
 'args': [<Dcel.Dcel at 0x7fc8d32ec2e0>, <Dcel.Dcel at 0x7fc8d32ef8b0>],
 'kwargs': {}}

In [12]:
print(res.target.listdir())

DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'Dcel.Dcel'>
['1', '2', '3', '4', '5']


In [13]:
type(None)

NoneType

In [14]:
print(res.target.inspect())
print(f"res.isdir(): {res.isdir()}")
print(f"res.target.isdir(): {res.target.isdir()}")

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class 'DictFS.DictFS'>:<DictFS.DictFS object at 0x7fc8d32ef9d0>
        value: <class 'Dcel.Dcel'>:{'1': {'spec': <Dcel.Dcel object at 0x7fc8d32ef850>, 'file': <Dcel.Dcel object at 0x7fc8d32ef3a0>, 'vfstype': <Dcel.Dcel object at 0x7fc8d32ef790>, 'mntopts': <Dcel.Dcel object at 0x7fc8d32ec430>, 'freq': <Dcel.Dcel object at 0x7fc8d32ef6a0>, 'passno': <Dcel.Dcel object at 0x7fc8d32ef490>}, '2': {'spec': <Dcel.Dcel object at 0x7fc8d32effa0>, 'file': <Dcel.Dcel object at 0x7fc8d32efe80>, 'vfstype': <Dcel.Dcel object at 0x7fc8d32efee0>, 'mntopts': <Dcel.Dcel object at 0x7fc8d32efd90>, 'freq': <Dcel.Dcel object at 0x7fc8d32effd0>, 'passno': <Dcel.Dcel object at 0x7fc8d32efa00>}, '3': {'spec': <Dcel.Dcel object at 0x7fc8d32efdf0>, 'file': <Dcel.Dcel object at 0x7fc8d32efe20>, 'vfstype': <Dcel.Dcel object at 0x7fc8d32efdc0>, 'mntopts': <Dcel.Dcel object at 0x7fc8d32eacb0>, 'freq': <Dcel.Dcel object at 0x7fc8d32eac50>, 

In [15]:
print(res.target.isdir())
print(hasattr(res.target.value,"__getitem__"))

True
True


In [16]:
print(res.target['1'])

{'1': {'spec': <Dcel.Dcel object at 0x7fc8d32ef850>, 'file': <Dcel.Dcel object at 0x7fc8d32ef3a0>, 'vfstype': <Dcel.Dcel object at 0x7fc8d32ef790>, 'mntopts': <Dcel.Dcel object at 0x7fc8d32ec430>, 'freq': <Dcel.Dcel object at 0x7fc8d32ef6a0>, 'passno': <Dcel.Dcel object at 0x7fc8d32ef490>}, '2': {'spec': <Dcel.Dcel object at 0x7fc8d32effa0>, 'file': <Dcel.Dcel object at 0x7fc8d32efe80>, 'vfstype': <Dcel.Dcel object at 0x7fc8d32efee0>, 'mntopts': <Dcel.Dcel object at 0x7fc8d32efd90>, 'freq': <Dcel.Dcel object at 0x7fc8d32effd0>, 'passno': <Dcel.Dcel object at 0x7fc8d32efa00>}, '3': {'spec': <Dcel.Dcel object at 0x7fc8d32efdf0>, 'file': <Dcel.Dcel object at 0x7fc8d32efe20>, 'vfstype': <Dcel.Dcel object at 0x7fc8d32efdc0>, 'mntopts': <Dcel.Dcel object at 0x7fc8d32eacb0>, 'freq': <Dcel.Dcel object at 0x7fc8d32eac50>, 'passno': <Dcel.Dcel object at 0x7fc8d32ea980>}, '4': {'spec': <Dcel.Dcel object at 0x7fc8d32ea6b0>, 'file': <Dcel.Dcel object at 0x7fc8d32ea860>, 'vfstype': <Dcel.Dcel object

In [17]:
print(res.target.value.inspect())

address: <class 'str'>:/
        abspath: <class 'str'>:/
        service: <class 'FormulaFS.FormulaFS'>:{"1": {"spec": "file://boot.localhost", "file": "{cburnuser}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=root,idcard=localuser", "freq": "0", "passno": "0"}, "2": {"spec": "file://fs2.cburn.io", "file": "{CBUSER}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=FishBo,idcard=localuser", "freq": "0", "passno": "0"}, "3": {"spec": "file://fs3.localhost", "file": "{cburnuser}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=Bucket9,idcard=localuser", "freq": "0", "passno": "0"}, "4": {"spec": "file://fs4.localhost", "file": "/", "vfstype": "cburnfs", "mntopts": "user,shortid=fs4,idcard=localuser", "freq": "0", "passno": "0"}, "5": {"spec": "http://metafs-redis", "file": "no-mount", "vfstype": "cburnfs-meta", "mntopts": "nouser,userid='raygan@raygan.com',userhome='raygan-home',userurl='https://raygan-raygan-com.home.laydbug.io'"}}
        value: <class 'd

In [27]:
resb = APath(Dcel(res.target, service_class=DictFS))
print(type(resb.target.value.value))
print(resb.target.value.isdir('/'))

DictFS::listdir(): path = /etc/cosmdirname
DictFS::listdir(): target type: <class 'list'>
DictFS::listdir(): path = /
DictFS::listdir(): target type: <class 'Dcel.Dcel'>
hiena_mp::type(text) is <class 'Dcel.Dcel'>
hiena_mp::type(text) is <class 'Dcel.Dcel'>
hiena_mp::type(text) is <class 'Dcel.Dcel'>
hiena_mp::type(text) is <class 'Dcel.Dcel'>
hiena_mp::type(text) is <class 'Dcel.Dcel'>
hiena_mp::type(text) is <class 'Dcel.Dcel'>
<class 'NoneType'>
False


## Test Other

In [ ]:
from MulticelSeqFS import MulticelSeqFS
from fs.base import FS
def fudgeglob(cel):
    return [cel[e] for e in cel.listdir()]
print(type(fudgeglob))
print(issubclass(MulticelSeqFS,FS))

fudgeglob(a.target)

In [ ]:
path = '.cosm/tools/cburn.app/img/IMG_5997.JPG'
b = a.lookup(path)
c = str(b.target)
c[0:10]

In [ ]:
a.listdir('/')

In [ ]:
urlstr='@/etc/fstab'
f = a.lookup(urlstr)
#d = Dcel(address=urlstr, service_class='file')
#f = APath(d)

g = f.command(dict,'fstab',f.target)
print(type(g.target))
print(g.target)
#print(g.target.__dict__)
print(g.listdir())
print(g.target['1']['spec'])

gcel = a.target
h = a.command(MulticelSeqFS,
              fudgeglob,
              [~gcel]
             )
print(h.listdir())


In [ ]:
urlstr = 'file://myserver/share/path'
d = APath(Dcel(value=urlstr))
b = d.command(dict, 'url',d.target)
print(type(b.target.value))
print(b.target.value['netloc'])

In [ ]:
print(b.target.value['netloc'])
b.listdir()

In [ ]:
from urllib.parse import urlparse

addrs = [ 
    'file://myserver/share/path',
    'file:path',
    '//myserver/share/path', 
    '/part/path'
    ]
for addr in addrs:
    url = urlparse(addr)
    print(url.path)


In [ ]:
from fs.opener import open as fsopen

addrs = ['fs','file://fs','file:fs']
for addr in addrs:
    print (fsopen(addr))

In [ ]:
a = [1,2,3]
b = [] 
c = [4,5,6]


a + b + c

In [ ]:
fstabs = a.target.path_lookup('.cosm/etc/fstab')
fstab_g = a.cosm.path_lookup('types/fstab')
b = a.command(dict,'fstab',fstabs)

In [ ]:
type(b.target.transformation_history) is Dcel

In [ ]:
print(b.target['1'])

In [ ]:
c = b.command(dict, 
              'tools/hiena', 
              b.target, 
              g2)
print(b.target.value)

In [ ]:
a.lookup('.cosm')

In [ ]:
c = a.getchild('.cosm/etc/fstab')

fstabs = c.readtext('.cosm/etc/fstab')
print(fstabs)

In [ ]:
h = a.cosm['tools']['hiena'].value
h

In [ ]:
g = { 'entry': r"([^\n]+)" }
g2 = { 'field': r"([^ ]+)" }

for e in h(fstabs,g)[0]['entry']:
    print(h(e,g2)[0]['field'])

In [ ]:
fstabd = [ h(e,g2)[0]['field']
          for e in h(fstabs,g)[0]['entry']]

print(fstabd)

In [ ]:
class FudgeBo(APathWrapper):
    pass

w = FudgeBo(a)
aa = APath(w)

aa is a


In [ ]:
d = D('.',OSFS)
ab = APath('fs','file',parent=a)

In [ ]:
ab.listdir('.')